In [2]:
import pandas as pd



In [3]:
index_file_path_template = '../data/input/TOEFL-iBT/Form {form_id:d}/writing_Form{form_id:d}.xlsx'
form1_fn = index_file_path_template.format(form_id=1)
form2_fn = index_file_path_template.format(form_id=2)

# print(form1_fn)
# Sample_ID, Domestic, Age, Gender, Country, Language, Reason, Institution_type, Parents_pay, Study_English, School_English, Lived_English, Integrated, Independent, Writing_raw_score

df1 = pd.read_excel(form1_fn)
df2 = pd.read_excel(form2_fn)

df1['Form'] = 1
df2['Form'] = 2
df = pd.concat([df1, df2], ignore_index=True)

df.head()

,Sample_ID,Domestic,Age,Gender,Country,Language,Reason,Institution_type,Parents_pay,Study_English,School_English,Lived_English,Integrated,Independent,Writing_raw_score,Form
0,20073241,1,26,M,JPN,JPN,2.0,3.0,4.0,6.0,5.0,3.0,4.0,3.0,7.0,1
1,20073242,1,34,M,TUR,TUR,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.0,6.5,1
2,20073243,1,26,F,TWN,CHI,2.0,3.0,NaN,NaN,NaN,1.0,2.5,2.0,4.5,1
3,20073244,1,33,NaN,ARG,SPA,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.5,4.5,1
4,20073245,2,23,M,IND,MAR,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.5,4.5,1


In [4]:
df1.shape, df2.shape

((240, 16), (240, 16))

In [21]:
df.value_counts(['Form', 'Independent']).sort_index()

Form  Independent
1     1.0             1
      1.5             3
      2.0            18
      2.5            32
      3.0            65
      3.5            40
      4.0            32
      4.5            35
      5.0            14
2     1.0             2
      1.5             2
      2.0            20
      2.5            25
      3.0            55
      3.5            45
      4.0            38
      4.5            28
      5.0            25
Name: count, dtype: int64

In [8]:
df_int_only = df1[df1['Independent'] % 1 == 0].copy()
df_int_only['Independent'] = df_int_only['Independent'].astype(int)
print(df_int_only['Independent'].value_counts())

Independent
3    65
4    32
2    18
5    14
1     1
Name: count, dtype: int64


In [18]:
value_counts_dict_df1 = df1['Independent'].value_counts().to_dict()
value_counts_dict_df2 = df2['Independent'].value_counts().to_dict()

print(value_counts_dict_df1, value_counts_dict_df2)

max_number_in_group = 10
# Combine dictionaries and calculate the minimum value for each key
combined_dict = {key: min(value_counts_dict_df1.get(key, 0), value_counts_dict_df2.get(key, 0), max_number_in_group)
                 for key in set(value_counts_dict_df1) | set(value_counts_dict_df2)}
combined_dict

{3.0: 65, 3.5: 40, 4.5: 35, 2.5: 32, 4.0: 32, 2.0: 18, 5.0: 14, 1.5: 3, 1.0: 1} {3.0: 55, 3.5: 45, 4.0: 38, 4.5: 28, 2.5: 25, 5.0: 25, 2.0: 20, 1.5: 2, 1.0: 2}


{1.5: 2, 2.5: 10, 3.0: 10, 3.5: 10, 4.5: 10, 4.0: 10, 2.0: 10, 5.0: 10, 1.0: 1}

In [19]:
# Sample N items from each value group in df1 based on the values in combined_dict
sampled_df1 = pd.concat([group.sample(combined_dict[key]) for key, group in df1.groupby('Independent')])
sampled_df1.value_counts('Independent')

Independent
2.0    10
2.5    10
3.0    10
3.5    10
4.0    10
4.5    10
5.0    10
1.5     2
1.0     1
Name: count, dtype: int64

In [29]:
# Define a custom function to sample a specified number of elements from each group
def custom_sample(group, num_elements):
    
    return group.sample(min(num_elements, len(group)), replace=False)

# Apply the custom function to each group with num_elements=2
df_sample = df1.groupby('Independent', group_keys=False).apply(custom_sample, num_elements=5)

# df_sample = df1.groupby('Independent', group_keys=False).sample(2)
df_sample['Independent'].value_counts()

Independent
2.0    5
2.5    5
3.0    5
3.5    5
4.0    5
4.5    5
5.0    5
1.5    3
1.0    1
Name: count, dtype: int64

In [18]:
df_disproportionate_sample = df1.groupby('Independent', group_keys=False).apply(lambda x: x.sample(frac=0.1))
df_disproportionate_sample['Independent'].value_counts()

Independent
3.0    6
3.5    4
4.5    4
2.5    3
4.0    3
2.0    2
5.0    1
Name: count, dtype: int64

,Sample_ID,Domestic,Age,Gender,Country,Language,Reason,Institution_type,Parents_pay,Study_English,School_English,Lived_English,Integrated,Independent,Writing_raw_score,Form
0,20073241,1,26,M,JPN,JPN,2.0,3.0,4.0,6.0,5.0,3.0,4.0,3.0,7.0,1
1,20073242,1,34,M,TUR,TUR,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.0,6.5,1
2,20073243,1,26,F,TWN,CHI,2.0,3.0,NaN,NaN,NaN,1.0,2.5,2.0,4.5,1
3,20073244,1,33,NaN,ARG,SPA,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.5,4.5,1
4,20073245,2,23,M,IND,MAR,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.5,4.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,20073235,2,18,F,GTM,SPA,1.0,1.0,NaN,5.0,3.0,1.0,4.0,4.5,8.5,1
236,20073233,1,26,F,IND,KOK,NaN,NaN,NaN,NaN,NaN,NaN,3.5,5.0,8.5,1
237,20073116,2,33,F,CHN,CHI,NaN,NaN,NaN,NaN,NaN,NaN,3.5,3.0,6.5,1
238,20073013,2,18,NaN,ALB,GRE,NaN,NaN,NaN,NaN,NaN,1.0,2.5,2.0,4.5,1
